#CBDE Lab 4 (part 1) - The Hadoop ecosystem (Hadoop & HBase)

#Tutorial

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##Setup environment

First, we will download and setup Hadoop and HBase. This notebook is configured to use the following versions of the systems:
*   Java JDK 8
*   Hadoop 3.3.6
*   HBase 2.5.6

In case some of the downloads fail (due to changes in versioning) the download link should be updated as well as all references to the version used.

### Hadoop

Download and install Hadoop

In [ ]:
#Setup JVM 8 (JVM 11 might cause incompatibility issues)
!apt-get update > /dev/null
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
# Download the latest version of Hadoop
!wget https://dlcdn.apache.org/hadoop/common/hadoop-3.3.6/hadoop-3.3.6.tar.gz
# Unzip it
# the tar command with the -x flag to extract, -z to uncompress, -v for verbose output, and -f to specify that we are extracting from a file
!tar -xzf hadoop-3.3.6.tar.gz
#copy hadoop file to user/local
!mv hadoop-3.3.6/ /usr/local/

--2023-11-10 17:51:38--  https://dlcdn.apache.org/hadoop/common/hadoop-3.3.6/hadoop-3.3.6.tar.gz
Resolving dlcdn.apache.org (dlcdn.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to dlcdn.apache.org (dlcdn.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 730107476 (696M) [application/x-gzip]
Saving to: ‘hadoop-3.3.6.tar.gz’

hadoop-3.3.6.tar.gz 100%[===================>] 696.28M   180MB/s    in 3.9s    

2023-11-10 17:51:42 (177 MB/s) - ‘hadoop-3.3.6.tar.gz’ saved [730107476/730107476]



Set environment variables

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["HADOOP_HOME"] = "/usr/local/hadoop-3.3.6/"

In [ ]:
# Add Hadoop BIN to PATH
current_path = os.getenv('PATH')
new_path = current_path+':/usr/local/hadoop-3.3.6/bin/'
os.environ["PATH"] = new_path

### HBase

Download and install HBase

In [ ]:
# Get the latest HBase version
!wget https://dlcdn.apache.org/hbase/2.5.6/hbase-2.5.6-bin.tar.gz
!tar xzf hbase-2.5.6-bin.tar.gz

--2023-11-10 17:56:14--  https://dlcdn.apache.org/hbase/2.5.6/hbase-2.5.6-bin.tar.gz
Resolving dlcdn.apache.org (dlcdn.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to dlcdn.apache.org (dlcdn.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 314596103 (300M) [application/x-gzip]
Saving to: ‘hbase-2.5.6-bin.tar.gz’

hbase-2.5.6-bin.tar 100%[===================>] 300.02M  21.0MB/s    in 5.4s    

2023-11-10 17:56:30 (55.9 MB/s) - ‘hbase-2.5.6-bin.tar.gz’ saved [314596103/314596103]



Set environment variables

In [ ]:
os.environ["HBASE_HOME"] = "/content/hbase-2.5.6/"
!echo $HBASE_HOME

/content/hbase-2.5.6/


In [ ]:
# Add HBase BIN to PATH
current_path = os.getenv('PATH')
new_path = current_path+':/content/hbase-2.5.6/bin'
os.environ["PATH"] = new_path
!echo $PATH

/opt/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin:/usr/local/hadoop-3.3.6/bin/:/content/hbase-2.5.6/bin


Verify all variables are correctly set

In [ ]:
!echo $JAVA_HOME
!echo $HADOOP_HOME
!echo $HBASE_HOME

/usr/lib/jvm/java-8-openjdk-amd64
/usr/local/hadoop-3.3.6/
/content/hbase-2.5.6/


For the purpose of this lab we will use an HBase standalone instance, thus no further configuration is required. If you were to use it in a cluster mode you would need to update the *hbase_site.xml* file

In [ ]:
# the file hbase-site.xml may need to be updated ...
#!cat $HBASE_HOME/conf/hbase-site.xml

### Start HBase server

In [ ]:
!start-hbase.sh
!jps

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/local/hadoop-3.3.6/share/hadoop/common/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hbase-2.5.6/lib/client-facing-thirdparty/log4j-slf4j-impl-2.17.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Reload4jLoggerFactory]
running master, logging to /content/hbase-2.5.6//logs/hbase--master-8e95860f6443.out
13812 Jps
13725 HMaster


# Bulk data loading

First, we are going to interact with HBase through the HBase shell. We will download a sample CSV file and load it into an HBase table.

See the following links for details on the available commands of the HBase shell:

*   https://www.tutorialspoint.com/hbase/hbase_shell.htm
*   https://www.guru99.com/hbase-shell-general-commands.html

We will download a publicly available file from the open repository of CSV files at https://people.sc.fsu.edu/~jburkardt/data/csv/csv.html. Precisely, we will download the *airtravel.csv* dataset, which contains monthly transatlantic airtravel, in thousands of passengers, for 1958-1960. There are 4 fields, "Month", "1958", "1959" and "1960" and 12 records, "JAN" through "DEC". There is also an initial header line.

In [ ]:
# Download sample file
!wget https://people.sc.fsu.edu/~jburkardt/data/csv/airtravel.csv

--2023-11-10 18:21:06--  https://people.sc.fsu.edu/~jburkardt/data/csv/airtravel.csv
Resolving people.sc.fsu.edu (people.sc.fsu.edu)... 144.174.0.22
Connecting to people.sc.fsu.edu (people.sc.fsu.edu)|144.174.0.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 321 [text/csv]
Saving to: ‘airtravel.csv’

airtravel.csv       100%[===================>]     321  --.-KB/s    in 0s      

2023-11-10 18:21:07 (133 MB/s) - ‘airtravel.csv’ saved [321/321]



In [ ]:
# Print its contents
!cat airtravel.csv

"Month", "1958", "1959", "1960"
"JAN",  340,  360,  417
"FEB",  318,  342,  391
"MAR",  362,  406,  419
"APR",  348,  396,  461
"MAY",  363,  420,  472
"JUN",  435,  472,  535
"JUL",  491,  548,  622
"AUG",  505,  559,  606
"SEP",  404,  463,  508
"OCT",  359,  407,  461
"NOV",  310,  362,  390
"DEC",  337,  405,  432



Then, using the HBase shell we will try to drop the table if it already exists. In case it does not exist the command will raise an error. To drop a table in HBase it is required first to **disable** it and then **drop** it.

In [ ]:
!echo "disable 'airtravel'" | hbase shell -n
!echo "drop 'airtravel'" | hbase shell -n

Then, we create the *airtravel* table. For demo purposes, we will define a single column family *years*, which will contain 3 qualifiers (one per year). The key of the table will be the month.

In [ ]:
!echo "create 'airtravel','years'" | hbase shell -n

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/local/hadoop-3.3.6/share/hadoop/common/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hbase-2.5.6/lib/client-facing-thirdparty/log4j-slf4j-impl-2.17.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Reload4jLoggerFactory]
2023-11-10 18:16:05,031 INFO  [main] Configuration.deprecation (Configuration.java:logDeprecation(1294)) - hbase.client.pause.cqtbe is deprecated. Instead, use hbase.client.pause.server.overloaded
2023-11-10 18:16:05,173 INFO  [ReadOnlyZKClient-127.0.0.1:2181@0x6f986501] zookeeper.ZooKeeper (Environment.java:logEnv(109)) - Client environment:zookeeper.version=3.5.7-f0fdd52973d373ffd9c86b81d99842dc2c7f660e, built on 02/10/2020 11:30 GMT
2023-11-10 18:16:05,174 INFO  [ReadOnlyZKClient-127

Using the **list** command we show the list of tables created in HBase.

In [ ]:
!echo "list" | hbase shell -n

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/local/hadoop-3.3.6/share/hadoop/common/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hbase-2.5.6/lib/client-facing-thirdparty/log4j-slf4j-impl-2.17.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Reload4jLoggerFactory]
2023-11-10 18:16:51,123 INFO  [main] Configuration.deprecation (Configuration.java:logDeprecation(1294)) - hbase.client.pause.cqtbe is deprecated. Instead, use hbase.client.pause.server.overloaded
2023-11-10 18:16:51,204 INFO  [ReadOnlyZKClient-127.0.0.1:2181@0x6f986501] zookeeper.ZooKeeper (Environment.java:logEnv(109)) - Client environment:zookeeper.version=3.5.7-f0fdd52973d373ffd9c86b81d99842dc2c7f660e, built on 02/10/2020 11:30 GMT
2023-11-10 18:16:51,204 INFO  [ReadOnlyZKClient-127

Next, to load the data in HBase we first need to move the CSV file from the local filesystem to the Hadoop filesystem. For that, we will use the **copyFromLocal** option of the **dfs** command. But first, using the **sed** command we willl remove the header of the CSV file (i.e., the first line).

In [ ]:
!sed -i '1d' airtravel.csv
!hdfs dfs -copyFromLocal airtravel.csv /tmp

Then, we will use the **ImportTsv** package (which is already provided by HBase) to bulk load a CSV file into an HBase table.

In [ ]:
!hbase org.apache.hadoop.hbase.mapreduce.ImportTsv -Dimporttsv.separator=',' -Dimporttsv.columns="HBASE_ROW_KEY,years:1958,years:1959,years:1960" airtravel /tmp/airtravel.csv

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/local/hadoop-3.3.6/share/hadoop/common/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hbase-2.5.6/lib/client-facing-thirdparty/log4j-slf4j-impl-2.17.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Reload4jLoggerFactory]
2023-11-10 18:24:12,926 INFO  [main] Configuration.deprecation (Configuration.java:logDeprecation(1294)) - hbase.client.pause.cqtbe is deprecated. Instead, use hbase.client.pause.server.overloaded
2023-11-10 18:24:13,114 INFO  [ReadOnlyZKClient-127.0.0.1:2181@0x0247d8ae] zookeeper.ZooKeeper (Environment.java:logEnv(109)) - Client environment:zookeeper.version=3.5.7-f0fdd52973d373ffd9c86b81d99842dc2c7f660e, built on 02/10/2020 11:30 GMT
2023-11-10 18:24:13,114 INFO  [ReadOnlyZKClient-127

Print the entire content of the table using the **scan** command (notice that the output is ordered lexicographically based on the key).

In [ ]:
!echo "scan 'airtravel'" | hbase shell -n

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/local/hadoop-3.3.6/share/hadoop/common/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hbase-2.5.6/lib/client-facing-thirdparty/log4j-slf4j-impl-2.17.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Reload4jLoggerFactory]
2023-11-10 18:25:25,588 INFO  [main] Configuration.deprecation (Configuration.java:logDeprecation(1294)) - hbase.client.pause.cqtbe is deprecated. Instead, use hbase.client.pause.server.overloaded
2023-11-10 18:25:25,685 INFO  [ReadOnlyZKClient-127.0.0.1:2181@0x6f986501] zookeeper.ZooKeeper (Environment.java:logEnv(109)) - Client environment:zookeeper.version=3.5.7-f0fdd52973d373ffd9c86b81d99842dc2c7f660e, built on 02/10/2020 11:30 GMT
2023-11-10 18:25:25,686 INFO  [ReadOnlyZKClient-127

# The HappyBase library

Here, we will showcase how to use the **HappyBase** (https://happybase.readthedocs.io/en/latest/) library to interact with HBase from Python. For that, we will also make use of HBase's Thrift server, which provides APIs to interact with HBase from programming languages beyond Java.

First, stop HBase in order to install HappyBase and start Thrift.

In [ ]:
!stop-hbase.sh

stopping hbase..............
SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/local/hadoop-3.3.6/share/hadoop/common/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hbase-2.5.6/lib/client-facing-thirdparty/log4j-slf4j-impl-2.17.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Reload4jLoggerFactory]


Install HappyBase

In [ ]:
!pip install happybase
import happybase

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 895.3 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.4/519.4 kB 6.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 5.8 MB/s eta 0:00:00
  Created wheel for happybase: filename=happybase-1.2.0-py2.py3-none-any.whl size=26602 sha256=27b079bccb8ba57a4b6351a2395963e239182f8d2dfd4f519a629693a010329a
  Stored in directory: /root/.cache/pip/wheels/95/d9/40/aef1e677ca6b53d419ead4f533d2a44198a1ce0b7b36b2437f
  Created wheel for thriftpy2: filename=thriftpy2-0.4.17-cp310-cp310-linux_x86_64.whl size=1382275 sha256=3963752ee0c4b03ac5883aa821c4af0240af08bb9e3e6383af1e32eb1bc9c1a2
  Stored in directory: /root/.cache/pip/wheels/b3/ca/c1/bd8e4cd94145e4e79882fdf705ca021a7a7632248ce

Start Thrift and HBasee

In [ ]:
!hbase-daemon.sh start thrift

running thrift, logging to /content/hbase-2.5.6//logs/hbase--thrift-8e95860f6443.out


In [ ]:
!start-hbase.sh
!jps

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/local/hadoop-3.3.6/share/hadoop/common/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hbase-2.5.6/lib/client-facing-thirdparty/log4j-slf4j-impl-2.17.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Reload4jLoggerFactory]
running master, logging to /content/hbase-2.5.6//logs/hbase--master-8e95860f6443.out
23825 ThriftServer
24242 Jps
24284 -- process information unavailable


### Interacting with HBase programatically

Create a connection object

In [ ]:
kxn = happybase.Connection('localhost',9090,autoconnect=False)
kxn

NameError: ignored

Display the tables (equivalent to the **list** command from the shell)

In [ ]:
kxn.open()
kxn.tables()

[b'airtravel']

Create a table

In [ ]:
kxn.open()
#Drop Table
kxn.disable_table('airtravel')
kxn.delete_table('airtravel')
#Create Table
kxn.create_table(
    'airtravel',
    {
        'years': dict()
    }
)
kxn.close()

NameError: ignored

Get the reference to the table

In [ ]:
kxn.open()
tAirtravel = kxn.table('airtravel')
kxn.close()

Put some data. Elements are inserted as key-value pairs, where the key must contain the column family and the qualifier names.

In [ ]:
kxn.open()
b = tAirtravel.batch()
b.put(b'JAN',{b'years:1958': b'340', b'years:1959': b'360', b'years:1960': b'417'})
b.put(b'FEB',{b'years:1958': b'318', b'years:1959': b'342', b'years:1960': b'391'})
b.put(b'MAR',{b'years:1958': b'362', b'years:1959': b'406', b'years:1960': b'419'})
b.send()
kxn.close()

Scan the complete table

In [ ]:
kxn.open()
for key, data in tAirtravel.scan():
    print(key, data)
kxn.close()

b'FEB' {b'years:1958': b'318', b'years:1959': b'342', b'years:1960': b'391'}
b'JAN' {b'years:1958': b'340', b'years:1959': b'360', b'years:1960': b'417'}
b'MAR' {b'years:1958': b'362', b'years:1959': b'406', b'years:1960': b'419'}


Scan the table from the key "FEB" to the key "JAN", here we specify the range via the **row_start** and **row_stop**. Note that the **row_start** is inclusive, while **row_stop** is exclusive. That is why if we want to include the key "JAN", we need to add some extra character.

In [ ]:
kxn.open()
for key, data in tAirtravel.scan(row_start=b'FEB', row_stop=b'JAN_'):
    print(key, data)
kxn.close()

b'FEB' {b'years:1958': b'318', b'years:1959': b'342', b'years:1960': b'391'}
b'JAN' {b'years:1958': b'340', b'years:1959': b'360', b'years:1960': b'417'}


Scan a row prefix

In [ ]:
kxn.open()
for key, data in tAirtravel.scan(row_prefix=b'M'):
    print(key, data)
kxn.close()

b'MAR' {b'years:1958': b'362', b'years:1959': b'406', b'years:1960': b'419'}


Scan a subset of column families

In [ ]:
kxn.open()
for key, data in tAirtravel.scan(columns=['years']):
    print(key, data)
kxn.close()

b'FEB' {b'years:1958': b'318', b'years:1959': b'342', b'years:1960': b'391'}
b'JAN' {b'years:1958': b'340', b'years:1959': b'360', b'years:1960': b'417'}
b'MAR' {b'years:1958': b'362', b'years:1959': b'406', b'years:1960': b'419'}


# On the key design

Key design is an essential aspect in HBase: i.e., decide how the row keys are going to be defined. Range queries can benefit from the B+ tree and the clustered index and drastically improve performance. As a simple example, imagine a query that uses a specific attribute to filter out the rows quite often and, accordingly, we decide that attribute to be the key. Since HBase indexes the key, we can use the index to perform the selections.
Key design is an important aspect of HBase (and in general, of many NOSQL database systems). So let us exemplify its relevance. Suppose a use case where data is generated every second and we decide to use the timestamp as row key in HBase.

In [ ]:
import time
import random

In [ ]:
kxn.open()
#kxn.disable_table('sensor_data')
#kxn.delete_table('sensor_data')
kxn.create_table(
    'sensor_data',
    {
        'measurements': dict()
    }
)
kxn.close()

In [ ]:
kxn.open()
tSensorData = kxn.table('sensor_data')
kxn.close()

In [ ]:
kxn.open()
b = tSensorData.batch()
for i in range(10):
  b.put(bytes(str(time.time()),'UTF-8'),
   {b'measurements:humidity':bytes(str(random.randint(0,100)),'UTF-8'),
    b'measurements:temperature':bytes(str(random.randint(0,30)),'UTF-8') })
  time.sleep(1)
b.send()
kxn.close()

In [ ]:
kxn.open()
for key, data in tSensorData.scan():
    print(key, data)
kxn.close()

b'1699641566.0903993' {b'measurements:humidity': b'15', b'measurements:temperature': b'28'}
b'1699641567.091501' {b'measurements:humidity': b'17', b'measurements:temperature': b'3'}
b'1699641568.0924923' {b'measurements:humidity': b'86', b'measurements:temperature': b'8'}
b'1699641569.0934975' {b'measurements:humidity': b'84', b'measurements:temperature': b'8'}
b'1699641570.0945022' {b'measurements:humidity': b'71', b'measurements:temperature': b'13'}
b'1699641571.0955389' {b'measurements:humidity': b'6', b'measurements:temperature': b'26'}
b'1699641572.0965' {b'measurements:humidity': b'70', b'measurements:temperature': b'30'}
b'1699641573.0976734' {b'measurements:humidity': b'27', b'measurements:temperature': b'29'}
b'1699641574.098819' {b'measurements:humidity': b'25', b'measurements:temperature': b'21'}
b'1699641575.0995371' {b'measurements:humidity': b'1', b'measurements:temperature': b'6'}


Do you think timestamp is a wise decision for the row key? Justify your answer.